In [96]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [87]:
dataset = pd.read_csv('data/fullDF.csv', index_col=0)
dataset = dataset.drop(['GRID_TYPE', 'PLAYER_ID', 'SHOT_ATTEMPTED_FLAG', 'TEAM_ID'], axis=1)

In [88]:
train_data.head()

,ACTION_TYPE,EVENTTIME,HTM,LOC_X,LOC_Y,MINUTES_REMAINING,PERIOD,PLAYER_NAME,QUARTER,SECONDS_REMAINING,SHOT_DISTANCE,SHOT_MADE_FLAG,SHOT_TIME,SHOT_TYPE,SHOT_ZONE_AREA,SHOT_ZONE_BASIC,SHOT_ZONE_RANGE,TEAM_NAME,VTM,PLAYER_AVG
70672,Jump Shot,60,OKC,170.6540,178.2374,1,2,Tony Snell,2.0,0,25,1,62.00,3PT Field Goal,Right Side Center(RC),Above the Break 3,24+ ft.,Chicago Bulls,CHI,0.398734
76585,Jump Shot,620,ATL,69.8612,230.1474,10,2,Jason Smith,2.0,20,18,0,611.80,2PT Field Goal,Right Side(R),Mid-Range,16-24 ft.,Orlando Magic,ORL,0.500000
60269,Turnaround Fadeaway shot,148,MEM,-20.2344,-2.5789,2,2,Marc Gasol,2.0,28,9,0,137.78,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,Memphis Grizzlies,DEN,0.455466
59583,Jump Shot,14,PHX,166.4626,181.1811,0,3,Mirza Teletovic,3.0,14,25,0,16.45,3PT Field Goal,Right Side Center(RC),Above the Break 3,24+ ft.,Phoenix Suns,DEN,0.433099
42833,Turnaround Jump Shot,25,CLE,118.7790,91.4692,0,4,Kristaps Porzingis,4.0,25,10,0,26.90,2PT Field Goal,Right Side(R),Mid-Range,8-16 ft.,New York Knicks,NYK,0.413866


In [90]:
categorical_cols = ['ACTION_TYPE', 'HTM', 'PERIOD', 'PLAYER_NAME', 'QUARTER', 'SHOT_TYPE', 'SHOT_ZONE_AREA',
                    'SHOT_ZONE_BASIC', 'SHOT_ZONE_RANGE', 'TEAM_NAME', 'VTM']

def onehot_encode(data):
    for feature in categorical_cols:
        one_hot = pd.get_dummies(data[feature], prefix=feature)
        data = data.drop(feature, axis = 1)
        data = data.join(one_hot)
    return data

dataset = onehot_encode(dataset)

In [92]:
train_data, test_data = train_test_split(dataset, test_size=0.2)
x_train = train_data.drop(['SHOT_MADE_FLAG'], axis=1)
y_train = train_data[['SHOT_MADE_FLAG']]
x_test = test_data.drop(['SHOT_MADE_FLAG'], axis=1)
y_test = test_data[['SHOT_MADE_FLAG']]

In [93]:
x_train.head()

,EVENTTIME,LOC_X,LOC_Y,MINUTES_REMAINING,SECONDS_REMAINING,SHOT_DISTANCE,SHOT_TIME,PLAYER_AVG,ACTION_TYPE_Alley Oop Dunk Shot,ACTION_TYPE_Alley Oop Layup shot,...,VTM_OKC,VTM_ORL,VTM_PHI,VTM_PHX,VTM_POR,VTM_SAC,VTM_SAS,VTM_TOR,VTM_UTA,VTM_WAS
31269,534,20.9447,8.4543,8,54,0,537.12,0.454874,0,0,...,0,0,0,0,0,0,0,0,0,0
81444,654,-102.8703,183.1123,10,54,18,657.99,0.480167,0,0,...,0,0,0,0,0,0,0,0,0,0
70719,527,160.2235,227.4563,8,47,9,564.03,0.508380,0,0,...,0,0,0,0,0,0,0,0,0,0
66009,688,130.1590,215.3484,11,28,2,681.00,0.475410,0,0,...,0,0,0,0,0,0,0,0,0,0
79753,628,-174.6799,41.3629,10,28,18,630.71,0.508380,0,0,...,1,0,0,0,0,0,0,0,0,0


In [94]:
model = Sequential()
model.add(Dense(1000, input_dim=len(x_train.columns), activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=80, batch_size=1024)

Epoch 1/80
67573/67573 [==============================] - 6s 85us/step - loss: 2.9425 - accuracy: 0.5239
Epoch 2/80
67573/67573 [==============================] - 5s 70us/step - loss: 0.6746 - accuracy: 0.5880
Epoch 3/80
67573/67573 [==============================] - 5s 70us/step - loss: 0.6709 - accuracy: 0.5994
Epoch 4/80
67573/67573 [==============================] - 5s 70us/step - loss: 0.6608 - accuracy: 0.6145
Epoch 5/80
67573/67573 [==============================] - 5s 69us/step - loss: 0.6608 - accuracy: 0.6129
Epoch 6/80
67573/67573 [==============================] - 5s 71us/step - loss: 0.6544 - accuracy: 0.6207
Epoch 7/80
67573/67573 [==============================] - 5s 72us/step - loss: 0.6553 - accuracy: 0.6210
Epoch 8/80
67573/67573 [==============================] - 5s 72us/step - loss: 0.6530 - accuracy: 0.6269
Epoch 9/80
67573/67573 [==============================] - 5s 73us/step - loss: 0.6385 - accuracy: 0.6377
Epoch 10/80
67573/67573 [==============================

67573/67573 [==============================] - 5s 80us/step - loss: 0.4912 - accuracy: 0.7408
Epoch 79/80
67573/67573 [==============================] - 5s 77us/step - loss: 0.4930 - accuracy: 0.7385
Epoch 80/80
67573/67573 [==============================] - 5s 77us/step - loss: 0.4845 - accuracy: 0.7440


In [95]:
y_test_pred = model.predict_classes(x_test)

In [98]:
print(f"accuracy: {round(accuracy_score(y_test, y_test_pred), 3)}")

accuracy: 0.62


In [100]:
model1 = Sequential()
model1.add(Dense(1000, input_dim=len(x_train.columns), activation='relu'))
model1.add(Dense(100, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))

model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model1.fit(x_train, y_train, epochs=120, batch_size=1024)

Epoch 1/120
67573/67573 [==============================] - 4s 65us/step - loss: 2.7159 - accuracy: 0.5378
Epoch 2/120
67573/67573 [==============================] - 3s 51us/step - loss: 0.6648 - accuracy: 0.6088
Epoch 3/120
67573/67573 [==============================] - 3s 51us/step - loss: 0.6679 - accuracy: 0.6107
Epoch 4/120
67573/67573 [==============================] - 3s 49us/step - loss: 0.6580 - accuracy: 0.6225
Epoch 5/120
67573/67573 [==============================] - 4s 56us/step - loss: 0.6785 - accuracy: 0.6100
Epoch 6/120
67573/67573 [==============================] - 4s 58us/step - loss: 0.6572 - accuracy: 0.6278
Epoch 7/120
67573/67573 [==============================] - 4s 58us/step - loss: 0.6516 - accuracy: 0.6307
Epoch 8/120
67573/67573 [==============================] - 4s 58us/step - loss: 0.6901 - accuracy: 0.6164
Epoch 9/120
67573/67573 [==============================] - 4s 62us/step - loss: 0.6457 - accuracy: 0.6353
Epoch 10/120
67573/67573 [====================

67573/67573 [==============================] - 3s 51us/step - loss: 0.4880 - accuracy: 0.7471
Epoch 78/120
67573/67573 [==============================] - 3s 51us/step - loss: 0.4862 - accuracy: 0.7477
Epoch 79/120
67573/67573 [==============================] - 4s 54us/step - loss: 0.4899 - accuracy: 0.7433
Epoch 80/120
67573/67573 [==============================] - 4s 53us/step - loss: 0.4833 - accuracy: 0.7503
Epoch 81/120
67573/67573 [==============================] - 4s 52us/step - loss: 0.4874 - accuracy: 0.7463
Epoch 82/120
67573/67573 [==============================] - 4s 54us/step - loss: 0.4803 - accuracy: 0.7509
Epoch 83/120
67573/67573 [==============================] - 4s 52us/step - loss: 0.4930 - accuracy: 0.7413
Epoch 84/120
67573/67573 [==============================] - 4s 53us/step - loss: 0.4749 - accuracy: 0.7565
Epoch 85/120
67573/67573 [==============================] - 4s 52us/step - loss: 0.4745 - accuracy: 0.7565
Epoch 86/120
67573/67573 [========================

In [101]:
y_test_pred_m1 = model1.predict_classes(x_test)
print(f"model 1 accuracy: {round(accuracy_score(y_test, y_test_pred_m1), 3)}")

model 1 accuracy: 0.607
